# 行列による逆伝播の演算

## ●重みの勾配を求める
逆伝播においては、以前に出力層の勾配や中間層の勾配を求めた際と同様に、まず$\delta$（デルタ）を求めます。  
$\delta$の求め方は層の種類や活性化関数の種類により異なりますが、これを行列$\Delta$で表すと以下のようになります。

$$  \begin{aligned} \\
\Delta & = \left(
    \begin{array}{cccc}
      \delta_{11} & \delta_{12} & \ldots & \delta_{1n} \\
      \delta_{21} & \delta_{22} & \ldots & \delta_{2n} \\
      \vdots & \vdots & \ddots & \vdots \\
      \delta_{h1} & \delta_{h2} & \ldots & \delta_{hn} \\
    \end{array}
  \right)
  \end{aligned}
 $$ 

$\Delta$は、層の出力$Y$と同じく、h x n、すなわち(バッチサイズ)x(ニューロン数)の行列になります。  

この$\delta$をもとに、各勾配を求めます。  
$\delta$の求め方以外は、出力層、中間層ともに各勾配の求め方は共通です。  
重みの勾配$\partial w_{ij}$を求めるためは、以前に導出した以下の式を用います。  

$$\partial w_{ij} = \frac{\partial E}{\partial w_{ij}} = y_i\delta_{j} $$

上の層の出力$y_i$は、この層のへの入力に等しいので、$y$は$x$に置き換えます。  
バッチに対応するためには、求めた勾配をバッチ内で足し合わせます。  
これは、以前に「エポックとバッチ」で解説した式をもとに、以下のように表すことができます。  

$$ \sum_{k=1}^{h} \frac{\partial E_k}{\partial w_{ij}}  $$

以上を踏まえて、バッチ対応した重みの勾配の行列$\partial W$は、次のようにして求めることができます。  

$$  \begin{aligned} \\
\partial W & = X^{\mathrm{T}}\Delta \\
& = \left(
    \begin{array}{cc}
      x_{11} & x_{21} & \ldots & x_{h1} \\
      x_{12} & x_{22} & \ldots & x_{h2} \\
      \vdots & \vdots & \ddots & \vdots \\
      x_{1m} & x_{2m} & \ldots & x_{hm} \\
    \end{array}
  \right)
\left(
    \begin{array}{cccc}
      \delta_{11} & \delta_{12} & \ldots & \delta_{1n} \\
      \delta_{21} & \delta_{22} & \ldots & \delta_{2n} \\
      \vdots & \vdots & \ddots & \vdots \\
      \delta_{h1} & \delta_{h2} & \ldots & \delta_{hn} \\
    \end{array}
  \right) \\
& = \left(
    \begin{array}{cc}
      \sum\limits_{k=1}^hx_{k1}\delta_{k1} & \sum\limits_{k=1}^hx_{k1}\delta_{k2} & \ldots & \sum\limits_{k=1}^hx_{k1}\delta_{kn} \\
      \sum\limits_{k=1}^hx_{k2}\delta_{k1} & \sum\limits_{k=1}^hx_{k2}\delta_{k2} & \ldots & \sum\limits_{k=1}^hx_{k2}\delta_{kn} \\
      \vdots & \vdots & \ddots & \vdots \\
      \sum\limits_{k=1}^hx_{km}\delta_{k1} & \sum\limits_{k=1}^hx_{km}\delta_{k2} & \ldots & \sum\limits_{k=1}^hx_{km}\delta_{kn} \\
    \end{array}
  \right)
  \end{aligned}
  $$ 

$X$と$\Delta$の行列積をとるためには、$X^{\mathrm{T}}$のように$X$を転置をして前の行列の列数と後ろの行列の行数を合わせる必要があります。  
結果的に得られた行列の各要素は、バッチ内で総和をとったものになっています。  
また、この行列のサイズはm x nであり、$W$のサイズと一致しています。 

以上は、次のようなシンプルコードで記述することができます。    

In [ ]:
grad_w = np.dot(x.T, delta)

`grad_w`は重みの勾配の行列$\partial W$で、`x`は入力の行列$X$、`delta`は$\delta$の行列$\Delta$を表します。

## ●バイアスの勾配を求める
バイアスの勾配は、以前に導出したバイアスの勾配の式を使って求めることができます。  

$$ \partial b_j = \delta_{j}$$ 

バッチ対応したバイアスの勾配は、次のように$\delta$をバッチ内で足し合わせることで求めることができます。

$$ \sum_{k=1}^{h} \frac{\partial E_k}{\partial b_j}  $$

これは、Numpyを用いて次のように実装することができます。  

In [ ]:
grad_b = np.sum(delta, axis=0)

`grad_b`はバイアスの勾配を表すベクトルです。  
Numpyのsum関数でaxisに0を指定すると、行列の縦方向で要素が足し合わされたベクトルを得ることができます。  
これにより、バッチ内で$\delta$を足し合わせることができます。  

## ●入力の勾配を求める
上の層の出力の勾配（この層の入力の勾配）を求めます。  
これは、上の層の$\delta$を求めるのに用いられます。  
以前に導出した式は次の通りです。  

（式 1）
$$ \partial y_j = \sum_{r=1}^n \delta_r w_{jr} $$

重みの勾配と同様に、上の層の出力$y_j$はこの層のへの入力に等しいので、$y$を$x$に置き換えます。  
そして、$\partial y$をバッチ対応した行列にしたものを、$\partial X$とします。   
これは次ように求めることができます。

$$  \begin{aligned} \\
\partial X & = \Delta W^{\mathrm{T}} \\
& = \left(
    \begin{array}{cccc}
      \delta_{11} & \delta_{12} & \ldots & \delta_{1n} \\
      \delta_{21} & \delta_{22} & \ldots & \delta_{2n} \\
      \vdots & \vdots & \ddots & \vdots \\
      \delta_{h1} & \delta_{h2} & \ldots & \delta_{hn} \\
    \end{array}
  \right) 
\left(
    \begin{array}{cccc}
      w_{11} & w_{21} & \ldots & w_{m1} \\
      w_{12} & w_{22} & \ldots & w_{m2} \\
      \vdots & \vdots & \ddots & \vdots \\
      w_{1n} & w_{2n} & \ldots & w_{mn} \\
    \end{array}
  \right)\\
& = \left(
    \begin{array}{cc}
      \sum\limits_{k=1}^n\delta_{1k}w_{1k} & \sum\limits_{k=1}^n\delta_{1k}w_{2k} & \ldots & \sum\limits_{k=1}^n\delta_{1k}w_{mk} \\
      \sum\limits_{k=1}^n\delta_{2k}w_{1k} & \sum\limits_{k=1}^n\delta_{2k}w_{2k} & \ldots & \sum\limits_{k=1}^n\delta_{2k}w_{mk} \\
      \vdots & \vdots & \ddots & \vdots \\
      \sum\limits_{k=1}^n\delta_{hk}w_{1k} & \sum\limits_{k=1}^n\delta_{hk}w_{2k} & \ldots & \sum\limits_{k=1}^n\delta_{hk}w_{mk}\\
    \end{array}
  \right)
  \end{aligned}
  $$ 

$\Delta$と$W$の行列積をとるために、$W^{\mathrm{T}}$のように$W$を転置します。  
これにより$\Delta$の列数と$W$の行数が一致し、行列積が計算できます。  
結果的に得られた行列の各要素は、この層のニューロンで総和をとったものになっており、（式 1）を表していることが分かります。

上記は、次のコードで記述することができます。  

In [ ]:
grad_x = np.dot(delta, w.T) 

`grad_x`は入力の勾配の行列$\partial X$です。  

以上により、逆伝播を行列の演算で表現することができました。